In [40]:
import pandas as pd
import numpy as np

test = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/test.csv')
train = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/train.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/gender_submission.csv')

In [41]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Baseline-модели
1. Предсказание cамого частотного класса / Предсказание средней метрики

In [6]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [7]:
submission['Survived'] = 0

In [9]:
submission.to_csv('2022_POL_baseline-1.csv', index=False)

2. Случайное предсказание

In [14]:
submission['Survived'] = np.random.choice([0, 1], len(submission))
submission.to_csv('2022_POL_random.csv', index=False)

3. Простое правило

1) Выжили только женщины

In [17]:
submission['Survived'] = test['Sex'].apply(lambda x: 1 if x =='female' else 0)
submission.to_csv('2022_POL_gender.csv', index=False)

2) Выжили только женщины и одинокие люди

In [34]:
test['Family'] = test['SibSp'] + test['Parch']

In [35]:
submission['Survived'] = np.where((test['Sex'] == 'female') | (test['Family'] == 0), 1,0)
submission.to_csv('2022_POL_gender_alone.csv', index=False)

In [39]:
submission['Survived']

0      1
1      1
2      1
3      1
4      1
      ..
413    1
414    1
415    1
416    1
417    0
Name: Survived, Length: 418, dtype: int64

In [56]:
def sep(name):
    return name.split(", ")[1].split(". ")[0]

test["Title"] = test["Name"].apply(sep)

test['Age'] = test['Age'].fillna(test.groupby('Title')['Age'].transform('mean'))
test['Age'] = test['Age'].fillna(test['Age'].mean())

In [60]:
train["Title"] = train["Name"].apply(sep)

train['Age'] = train['Age'].fillna(train.groupby('Title')['Age'].transform('mean'))
train['Age'] = train['Age'].fillna(train['Age'].mean())

In [63]:
X_train = train[['Age', 'SibSp', 'Parch']]
X_test = test[['Age', 'SibSp', 'Parch']]
y_train = train['Survived']

In [64]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [67]:
submission['Survived'] = model.predict(X_test)
submission.to_csv('dt-1.csv', index=False)

In [68]:
model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [69]:
model = DecisionTreeClassifier(max_leaf_nodes = 10)
model.fit(X_train, y_train)

submission['Survived'] = model.predict(X_test)
submission.to_csv('dt-2.csv', index=False)